<a href="https://colab.research.google.com/github/bodamohannaik/DLAI-TF-DPC/blob/master/C3/W2/C3_W2_sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

# Download Dataset

In [2]:
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2022-05-15 03:53:52--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.135.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  33.7MB/s    in 0.2s    

2022-05-15 03:53:52 (33.7 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]



# Preprocess Data

In [3]:
with open('sarcasm.json', 'r') as fp:
  sarcasm = json.load(fp)
len(sarcasm)

26709

In [4]:
sarcasm[0]

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
 'is_sarcastic': 0}

In [5]:
links = []
sentences = []
labels = []
for item in sarcasm:
  links.append(item['article_link'])
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])
labels = np.array(labels)

In [6]:
list(zip(sentences[:5], labels))

[("former versace store clerk sues over secret 'black code' for minority shoppers",
  0),
 ("the 'roseanne' revival catches up to our thorny political mood, for better and worse",
  0),
 ("mom starting to fear son's web series closest thing she will have to grandchild",
  1),
 ('boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
  1),
 ('j.k. rowling wishes snape happy birthday in the most magical way', 0)]

## Train Test Split

In [7]:
from sklearn.model_selection import train_test_split
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=.2)
len(train_sentences), len(test_sentences)

(21367, 5342)

## Tokenizer

In [8]:
num_words = 10000
oov_token = "<OOV>"
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, lower=True, split=' ', oov_token=oov_token)
tokenizer.fit_on_texts(texts=train_sentences)

In [9]:
len(tokenizer.word_counts)

26591

In [10]:
train_sequences = tokenizer.texts_to_sequences(texts = train_sentences)
test_sequences = tokenizer.texts_to_sequences(texts = test_sentences)

In [11]:
maxlen = 32
padding = "post"
truncating = "post"
train_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences=train_sequences, maxlen=maxlen, padding=padding, truncating=truncating)
test_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences=test_sequences, maxlen=maxlen, padding=padding, truncating=truncating)

In [12]:
train_sequences.shape

(21367, 32)

In [13]:
test_sequences.shape

(5342, 32)

# Neural Network

In [21]:
model = tf.keras.Sequential(layers=[
                                    tf.keras.layers.Input(shape=(maxlen,)),
                                    tf.keras.layers.Embedding(input_dim = num_words, output_dim=8),
                                    tf.keras.layers.GlobalAveragePooling1D(),
                                    # tf.keras.layers.Dense(units=8, activation='relu'),
                                    tf.keras.layers.Dropout(rate = .2),
                                    tf.keras.layers.Dense(units=1, activation='sigmoid')
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 32, 8)             80000     
                                                                 
 global_average_pooling1d_4   (None, 8)                0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_4 (Dropout)         (None, 8)                 0         
                                                                 
 dense_6 (Dense)             (None, 1)                 9         
                                                                 
Total params: 80,009
Trainable params: 80,009
Non-trainable params: 0
_________________________________________________________________


In [22]:
# compile
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# train
history = model.fit(x=train_sequences, y=train_labels, batch_size=16, epochs=10, validation_data = (test_sequences, test_labels))

Epoch 1/10
1336/1336 [==============================] - 6s 4ms/step - loss: 0.6493 - accuracy: 0.6069 - val_loss: 0.5839 - val_accuracy: 0.7630
Epoch 2/10
1336/1336 [==============================] - 5s 4ms/step - loss: 0.5026 - accuracy: 0.8172 - val_loss: 0.4525 - val_accuracy: 0.8205
Epoch 3/10
1336/1336 [==============================] - 5s 4ms/step - loss: 0.3984 - accuracy: 0.8560 - val_loss: 0.3953 - val_accuracy: 0.8355
Epoch 4/10
1336/1336 [==============================] - 5s 4ms/step - loss: 0.3410 - accuracy: 0.8749 - val_loss: 0.3652 - val_accuracy: 0.8433
Epoch 5/10
1336/1336 [==============================] - 5s 4ms/step - loss: 0.3019 - accuracy: 0.8887 - val_loss: 0.3474 - val_accuracy: 0.8504
Epoch 6/10
1336/1336 [==============================] - 5s 4ms/step - loss: 0.2726 - accuracy: 0.9008 - val_loss: 0.3371 - val_accuracy: 0.8519
Epoch 7/10
1336/1336 [==============================] - 5s 4ms/step - loss: 0.2492 - accuracy: 0.9098 - val_loss: 0.3312 - val_accuracy: